<a href="https://colab.research.google.com/github/NahyeonKang/classify-spending/blob/main/build_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. 음식점
1. 카페
2. 식재료
3. 편의점
5. 뷰티
6. 패션
7. 의료
8. 여행
9. 공연/영화
10. 스포츠
11. 기타 쇼핑
12. 간편결제
13. 교육비
14. 교통비
15. 생활요금
16. 기타


In [1]:
import pandas as pd

data = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/data.xlsx", header=None)

In [3]:
data['category'] = "0"

In [4]:
data.columns=['document', 'label']

In [6]:
data_test = data.loc[213:]

In [8]:
data_test.to_csv("data_test.csv")

# GPT4 API로 분류하기

In [ ]:
! pip install openai

In [ ]:
import openai

openai.api_key = ""

In [ ]:
for i in range(166, 664):
  spending = data.loc[i]['spending']

  messages = [
      {"role":"system", "content": "Categorize a given <spending> into the following categories. \n0.restaurant\n1.cafe\n2.grocery\n3.convenience store\n4.cosmetics\n5.fashion\n6.healthcare\n7.travel\n8.shows/movies\n9.sports\n10.other shoppings\n11.payment service\n12.education\n13.transportation\n14.utility bills\n15.Others  "},
      {"role":"user", "content":f"<spending>\n{spending}"}
  ]

  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages
  )

  answer = response['choices'][0]['message']['content']
  data.loc[i]['category']=answer

APIError: ignored

In [ ]:
i

213

In [ ]:
data.to_excel("/content/drive/MyDrive/Colab Notebooks/GPTlabeleddata.xlsx")

# GPT4 답변 & 크롤링 데이터로 huggingface Dataset 만들기

In [ ]:
! pip install datasets

In [3]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': "/content/drive/MyDrive/Colab Notebooks/train_labler_data.csv"})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'label'],
        num_rows: 3475
    })
})

# BERT 학습하기

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
!pip install accelerate -U

In [7]:
import random
import os
import numpy as np
import torch

def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # for faster training, but not deterministic

set_seeds(seed=42)

In [8]:
from huggingface_hub import notebook_login

notebook_login()

## num_labels 추가

In [9]:
from transformers import AutoTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = BertForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=16).to('cuda')

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['encoder.layer.1.output.dense.bias', 'encoder.layer.6.attention.self.value.weight', 'encoder.layer.11.output.LayerNorm.bias', 'encoder.layer.9.attention.output.LayerNorm.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.attention.self.key.weight', 'encoder.layer.3.attention.self.key.bias', 'encoder.layer.1.attention.self.key.bias', 'encoder.layer.5.attention.self.query.bias', 'embeddings.word_embeddings.weight', 'encoder.layer.11.attention.self.key.bias', 'encoder.layer.4.attention.self.query.bias', 'encoder.layer.7.attention.self.query.bias', 'encoder.layer.5.output.LayerNorm.bias', 'encoder.layer.7.attention.output.dense.bias', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.6.output.dense.weight', 'embeddings.LayerNorm.weight', 'encoder.layer.7.attention.self.value.bias', 

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["document"], truncation=True)

In [11]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3475 [00:00<?, ? examples/s]

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
import evaluate

accuracy = evaluate.load("accuracy")

In [14]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## wandb로 로깅

In [ ]:
!pip install wandb

In [16]:
# Import wandb
import wandb


# Login with your authentication key
wandb.login()


# setup wandb environment variables
%env WANDB_ENTITY='nahyeonkang/ai-keepit'
%env WANDB_PROJECT='classify-spending'


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_ENTITY='nahyeonkang/ai-keepit'
env: WANDB_PROJECT='classify-spending'


In [22]:
import torch
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="ai.keepit/classifier",
    learning_rate=5e-6,
    lr_scheduler_type="cosine",
    per_device_train_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
    report_to="wandb",
    run_name="klue-roberta-base-10epoch-5e6"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
500,0.704900
1000,0.549100
1500,0.443700
2000,0.363500


TrainOutput(global_step=2180, training_loss=0.5010664511164394, metrics={'train_runtime': 237.9733, 'train_samples_per_second': 146.025, 'train_steps_per_second': 9.161, 'total_flos': 209752595999616.0, 'train_loss': 0.5010664511164394, 'epoch': 10.0})

In [23]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

'https://huggingface.co/nahyeonkang/classifier/tree/main/'

# Inference

In [30]:
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
import pandas as pd

pipe = pipeline("text-classification", model="nahyeonkang/classifier", device=0)
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data_test.csv")

In [35]:
test_df

,document,label
0,포토이즘박스 연,0
1,김밥마는아저씨,0
2,피플PC방,0
3,까치네분식,0
4,3공장,0
...,...,...
446,금고ATM입금,0
447,가시버시갈비탕.찜,0
448,펀비어킹,0
449,한국철도공사,0


In [42]:
for i in range(451):
  answer = pipe(test_df.iloc[i]['document'])
  if answer[0]['score'] < 0.5:
    test_df.loc[i, 'label'] = 'LABEL_15'
  else:
    test_df.loc[i, 'label'] = answer[0]['label']
  print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'label': 'LABEL_4', 'score': 0.809537947177887}]
[{'label': 'LABEL_8', 'score': 0.3432818353176117}]
[{'label': 'LABEL_5', 'score': 0.7972341775894165}]
[{'label': 'LABEL_0', 'score': 0.610237181186676}]
[{'label': 'LABEL_9', 'score': 0.6232358813285828}]
[{'label': 'LABEL_9', 'score': 0.852716863155365}]
[{'label': 'LABEL_5', 'score': 0.7285472750663757}]
[{'label': 'LABEL_5', 'score': 0.6484419107437134}]
[{'label': 'LABEL_1', 'score': 0.7772951126098633}]
[{'label': 'LABEL_0', 'score': 0.489682674407959}]
[{'label': 'LABEL_1', 'score': 0.7473235130310059}]
[{'label': 'LABEL_5', 'score': 0.4040205180644989}]
[{'label': 'LABEL_5', 'score': 0.8375521898269653}]
[{'label': 'LABEL_5', 'score': 0.8520181179046631}]
[{'label': 'LABEL_5', 'score': 0.5708414912223816}]
[{'label': 'LABEL_8', 'score': 0.7136379480361938}]
[{'label': 'LABEL_1', 'score': 0.8675923347473145}]
[{'label': 'LABEL_5', 'score': 0.4460678696632385}]
[{'label': 'LABEL_9', 'score': 0.4759807288646698}]
[{'label': 'LABE

In [44]:
test_df['label'].value_counts()

LABEL_15    137
LABEL_5     121
LABEL_0      48
LABEL_9      33
LABEL_1      29
LABEL_4      18
LABEL_8      16
LABEL_12     13
LABEL_6      12
LABEL_2      12
LABEL_11      9
LABEL_7       3
Name: label, dtype: int64

In [45]:
test_df.to_excel("/content/drive/MyDrive/Colab Notebooks/labeled.xlsx")